## Goal: In a different notebook, kaiserOptimizationWithoutCcd, ascvd, non cv death, and dementia intercepts were optimized to represent the average increased risk of the Kaiser population without taking into account WMH-specific information (eg hazard ratios for WMH severity, SBI etc). The goal of this notebook is to test whether the optimization was done correctly. The outcome incidence rates we obtain here are very close to the reference rates (note: the reference rates were obtained from a different simulation/notebook).

In [1]:
import numpy as np
import pandas as pd
import os
import importlib.util
from scipy import optimize
import matplotlib.pyplot as plt

microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/CODE/microsim"
os.chdir(microsimDir)
dataDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/NOTEBOOKS/DATA"

pd.set_option('future.no_silent_downcasting', True) 

from microsim.outcome import OutcomeType
from microsim.population_factory import PopulationFactory
from microsim.dementia_model_repository import DementiaModelRepository
from microsim.dementia_model import DementiaModel
from microsim.cv_model import CVModelMale, CVModelFemale
from microsim.cv_model_repository import CVModelRepository
from microsim.non_cv_death_model import NonCVDeathModel

### Part 1: validate that the mean risk, in the Kaiser population, is the same using either the SCD-specific or the non-SCD specific ASCVD model.

In [2]:
popSize = 300000
pop = PopulationFactory.get_kaiser_population(n=popSize)

In [3]:
cvNonScdSpecificRiskList = list(map(lambda x: CVModelRepository(wmhSpecific=False).select_outcome_model_for_person(x).get_risk_for_person(x, years=10),
                                            pop._people))

cvRiskList = list(map(lambda x: CVModelRepository().select_outcome_model_for_person(x).get_risk_for_person(x, years=10), pop._people))

In [4]:
meanNonScdSpecificRisk = sum(cvNonScdSpecificRiskList)/len(cvNonScdSpecificRiskList)
meanScdSpecificRisk = sum(cvRiskList)/len(cvRiskList)
print(f"Mean risk without taking into account SCD-specific information: {meanNonScdSpecificRisk:4.3f}")
print(f"Mean risk with taking into account SCD-specific information: {meanScdSpecificRisk:4.3f}")
print(f"Percent absolute difference: {abs(meanNonScdSpecificRisk-meanScdSpecificRisk)/meanScdSpecificRisk:4.3f}")

Mean risk without taking into account SCD-specific information: 0.233
Mean risk with taking into account SCD-specific information: 0.245
Percent absolute difference: 0.050


### Part 2: validate the outcome incidence rates, using as the reference the outcome incidence rates obtained by running a Microsim simulation with the SCD-specific models.

In [5]:
class DementiaModelRepositoryKaiser(DementiaModelRepository):
    def __init__(self, wmhSpecific=False):
        super().__init__(wmhSpecific=False)
        
module = importlib.import_module("microsim.outcome_model_repository")
setattr(module, "DementiaModelRepository", DementiaModelRepositoryKaiser)

In [6]:
class NonCVDeathModelKaiser(NonCVDeathModel):
    def __init__(self, wmhSpecific):
            super().__init__(wmhSpecific=False)

module = importlib.import_module("microsim.non_cv_model_repository")
setattr(module, "NonCVDeathModel", NonCVDeathModelKaiser)

In [7]:
class CVModelMaleKaiser(CVModelMale):
    def __init__(self, wmhSpecific):
            super().__init__(wmhSpecific=False)

class CVModelFemaleKaiser(CVModelFemale):
    def __init__(self, wmhSpecific):
            super().__init__(wmhSpecific=False)

module = importlib.import_module("microsim.cv_model_repository")
setattr(module, "CVModelMale", CVModelMaleKaiser)
setattr(module, "CVModelFemale", CVModelFemaleKaiser)

In [8]:
#because the population was created prior to the model swapping above, it has SCD-specific models, eg
pop._modelRepository["outcomes"]._repository[OutcomeType.DEMENTIA]._models["brainScan"].wmhSpecific

True

In [9]:
#so I need to re-create the population now
pop = PopulationFactory.get_kaiser_population(n=popSize)

In [10]:
#and check again that the population has only non-SCD-specific models
(pop._modelRepository["outcomes"]._repository[OutcomeType.DEMENTIA]._models["brainScan"].wmhSpecific,
pop._modelRepository["outcomes"]._repository[OutcomeType.NONCARDIOVASCULAR]._model.wmhSpecific,
pop._modelRepository["outcomes"]._repository[OutcomeType.CARDIOVASCULAR]._models["male"].wmhSpecific,
pop._modelRepository["outcomes"]._repository[OutcomeType.CARDIOVASCULAR]._models["female"].wmhSpecific)

(False, False, False, False)

In [11]:
%%time
duration=4
pop.advance(duration, nWorkers=1)

CPU times: user 19min 21s, sys: 5.47 s, total: 19min 26s
Wall time: 19min 26s


In [12]:
strokeRates = pop.get_outcome_incidence_rates_at_end_of_wave(outcomesTypeList=[OutcomeType.STROKE], 
                                                                                     wave=duration-1)
dementiaRates = pop.get_outcome_incidence_rates_at_end_of_wave(outcomesTypeList=[OutcomeType.DEMENTIA], 
                                                                                       wave=duration-1)
deathRates = pop.get_outcome_incidence_rates_at_end_of_wave(outcomesTypeList=[OutcomeType.DEATH], 
                                                                                    wave=duration-1)
miRates = pop.get_outcome_incidence_rates_at_end_of_wave(outcomesTypeList=[OutcomeType.MI], 
                                                                                     wave=duration-1)
rates = {"stroke": strokeRates, "dementia": dementiaRates, "death": deathRates, "mi": miRates}

In [13]:
strokeRatesRef = 12 #microsim simulation

deathRatesRef = 27 #microsim simulation

dementiaRatesRef = 11 #microsim simulation

ratesRef = {"stroke": strokeRatesRef, "death": deathRatesRef, "dementia": dementiaRatesRef, "mi": 12}

In [18]:
print("\nReference: Microsim simulation with SCD-specific models\n")
print("Simulation: Microsim simulation with non-SCD-specific models\n")
print("     Group     Reference     Simulation")
for outcome in rates.keys():
    print(f"{outcome:>10}" +
        f"{ratesRef[outcome]:>14.1f} " + 
          f"{rates[outcome]:>14.1f}")


Reference: Microsim simulation with SCD-specific models

Simulation: Microsim simulation with non-SCD-specific models

     Group     Reference     Simulation
    stroke          12.0           11.6
  dementia          11.0           10.8
     death          27.0           26.7
        mi          12.0           11.6
